In [3]:
import os
import re

import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
save_dir = '/diskmnt/Projects/Users/estorrs/sc_pdac/results/supplemental_figures'
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [5]:
clinical = pd.read_csv('../data/pdac/eus_clinical_formatted.txt', sep='\t', index_col=0)
clinical

,Patient/Sample ID,1-liner,Sample/Biopsy Date,Gender,Age,Ethnicity,Patient being followed at Wash U?,Stage (AJCC 8),T,N,...,Dz Progression?,Prog date or last imaging f/u,PFS time (mo),Died?,Death date or last clinical f/u,OS time (mo),Notes,Unnamed: 42,overall_survival,status
wu01,WU001,NaN,2/4/19,Female,61.0,White,No,NaN,3.0,0,...,peritoneal metastasis,2/5/19,NaN,Yes,4/9/19,NaN,No clinical follow up at WashU,NaN,2.0,Dead
wu02,WU002,NaN,2/4/19,Female,90.0,White,No,NaN,3.0,1,...,mass tethers the small bowel,2/21/19,NaN,Yes,4/3/19,NaN,Final note in pt's chart states to cancel all ...,NaN,1.0,Dead
wu03,WU003,NaN,3/5/19,Female,71.0,White,No,NaN,2.0,1,...,enlarged nodes suspicious for mets,3/11/19,NaN,Yes,4/3/19,NaN,Last note in pt's chart states pt had stroke a...,NaN,0.0,Dead
wu04,WU004,NaN,3/22/19,Male,72.0,White,Yes,NaN,2.0,1,...,NaN,10/16/19,NaN,Yes,12/9/20,NaN,Final note says patient passed away in his hom...,NaN,20.0,Dead
wu05,WU005,NaN,3/27/19,Female,74.0,White,Yes,NaN,3.0,0,...,no evidence of residual mets,10/14/19,NaN,NaN,3/4/22,NaN,Still alive as of 3/4/22,NaN,35.0,Alive
wu06,WU006,NaN,3/29/19,Female,60.0,White,Yes,NaN,2.0,1,...,"no, stable",10/22/19,NaN,Yes,3/18/20,NaN,Encounter in chart shows that patient passed a...,NaN,11.0,Dead
wu07,WU007,NaN,5/3/19,Male,80.0,White,No,NaN,3.0,0,...,terminal,10/1/19,NaN,Yes,10/1/19,NaN,Pt died at home on hospice,NaN,5.0,Dead
wu08,WU008,NaN,5/14/19,Female,71.0,White,No,NaN,1.0,0,...,NaN,6/13/19,NaN,Yes,8/10/19,NaN,No follow up at WashU,NaN,2.0,Dead
wu09,WU009,NaN,5/17/19,Female,80.0,White,Yes,NaN,2.0,1,...,No mets,7/29/19,NaN,Yes,1/14/20,NaN,NaN,NaN,8.0,Dead
wu10,WU010,NaN,8/1/19,Male,66.0,White,Yes,NaN,2.0,x,...,Stable,8/23/19,NaN,NaN,3/4/22,NaN,Still alive as of 3/4/22,NaN,31.0,Alive


In [6]:
clinical.to_csv(os.path.join(save_dir, 'EUS_FNB_formatted_clinical_metadata.txt'), sep='\t', index=False)

In [7]:
adata = sc.read_h5ad('../backups/final_adata.h5ad')
adata

AnnData object with n_obs × n_vars = 140612 × 30840
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.1.25', 'seurat_clusters', 'integrated_snn_res.2', 'integrated_snn_res.3', 'integrated_snn_res.1', 'integrated_snn_res.0.5', 'active.ident', 'cell_type', 'dataset', 'sample_id', 'cell_type_level_2', 'cell_type_level_3', 'tumor_original_seurat', 'malignant_exclude', 'final_cell_types'
    var: 'is_highly_variable'
    uns: 'cell_type_colors', 'cell_type_level_2_colors', 'cell_type_level_3_colors', 'dataset_colors', 'final_cell_types_colors', 'rank_genes_groups'
    obsm: 'X_umap'
    layers: 'counts'

In [8]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,CELL,CONDITION,Patient,Type,Cell_type,...,integrated_snn_res.0.5,active.ident,cell_type,dataset,sample_id,cell_type_level_2,cell_type_level_3,tumor_original_seurat,malignant_exclude,final_cell_types
wu01_AAACCTGCACATAACC-1,wu01,11822.0,3295,3377.0,1314,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACCTGCACCACGTG-1,wu01,469.0,293,2235.0,450,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACCTGTCTAACTTC-1,wu01,482.0,243,2436.0,400,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,0,Malignant,Malignant,In-house: EUS-FNB,wu01,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low
wu01_AAACGGGCAAGGCTCC-1,wu01,1516.0,717,2944.0,717,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,2,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
wu01_AAACGGGTCATTCACT-1,wu01,876.0,453,2491.0,524,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,14,Monocyte,Monocyte,In-house: EUS-FNB,wu01,Monocyte,Monocyte,Monocyte,False,Monocyte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGGTTTCTACCAGA-1,G9903,8237.0,2326,9379.0,2326,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,1,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,G9903,6484.0,1875,9150.0,1886,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,1,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACAACTTGAC-1,G9903,8367.0,2286,9400.0,2286,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,1,CAF,CAF,Chan-Seng-Yue,G9903,CAF,CAF,CAF,False,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCATCCGAACGC-1,G9903,20314.0,3975,10684.0,3464,NA_character_,-2147483648,NA_character_,-2147483648,NA_character_,...,0,Malignant,Malignant,Chan-Seng-Yue,G9903,Malignant_0,Malignant_0,Malignant_0,False,Malignant - Classical Low


In [11]:
keep = ['dataset', 'sample_id', 'final_cell_types']
adata.obs = adata.obs[keep]
adata.obs

,dataset,sample_id,final_cell_types
wu01_AAACCTGCACATAACC-1,In-house: EUS-FNB,wu01,Monocyte
wu01_AAACCTGCACCACGTG-1,In-house: EUS-FNB,wu01,Monocyte
wu01_AAACCTGTCTAACTTC-1,In-house: EUS-FNB,wu01,Malignant - Classical Low
wu01_AAACGGGCAAGGCTCC-1,In-house: EUS-FNB,wu01,Monocyte
wu01_AAACGGGTCATTCACT-1,In-house: EUS-FNB,wu01,Monocyte
...,...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGGTTTCTACCAGA-1,Chan-Seng-Yue,G9903,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,Chan-Seng-Yue,G9903,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCACAACTTGAC-1,Chan-Seng-Yue,G9903,CAF
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCATCCGAACGC-1,Chan-Seng-Yue,G9903,Malignant - Classical Low


In [14]:
for k in list(adata.uns.keys()):
    adata.uns.pop(k)

In [15]:
adata

AnnData object with n_obs × n_vars = 140612 × 30840
    obs: 'dataset', 'sample_id', 'final_cell_types'
    var: 'is_highly_variable'
    obsm: 'X_umap'
    layers: 'counts'

In [16]:
adata.write_h5ad(os.path.join(save_dir, 'integrated_single_cell_adata.h5ad'))

In [98]:
from collections import Counter
Counter(adata.obs['dataset'])

Counter({'In-house: EUS-FNB': 28313,
         'In-house: Surgical': 30245,
         'Peng': 41440,
         'Lin': 8535,
         'Chan-Seng-Yue': 32079})

In [100]:
len(set(adata.obs['sample_id']))

77

In [101]:
gene_set_map = {
    'Bailey': {
        'ADEX': [
            "MTMR7",
            "ARX",
            "ABCC8",
            "CEACAM7",
            "CACNA2D2",
            "MAPK8IP2",
            "SYT7",
            "LTF",
            "NR1H4",
            "SLC7A14",
            "PRSS3",
            "CPA1",
            "AQP8",
            "FGL1",
            "SERPINI2",
            "REG1A",
            "NR5A2",
            "RBPJL",
            "KIRREL2",
            "CLP",
        ],
        'Squamous': [
            "CFH",
            "CD99",
            "HECW1",
            "PLXND1",
            "RECQL",
            "C1orf112",
            "M6PR",
            "FKBP4",
            "GGCT",
            "DBF4",
            "SEMA3F",
            "ANKIB1",
            "MAD1L1",
            "CDC27",
            "WDR54",
            "DPM1",
            "NME2",
            "UTP18",
            "SLC25A39",
            "TTC2",
        ],
        'Pancreatic Progenitor': [
            "ENPP4",
            "CFTR",
            "CYP51A1",
            "ABP1",
            "LASP1",
            "TMEM176A",
            "ICA1",
            "DBNDD1",
            "CASP10",
            "SARM1",
            "UPF1",
            "ACSM3",
            "SPPL2B",
            "PDK2",
            "OSBPL7",
            "TMEM98",
            "BAIAP2L1",
            "ALDH3B1",
            "TTC22",
            "FARP",    
        ],
        'Immunogenic': [
            "CD38",
            "PDK4",
            "FBXL3",
            "CD79B",
            "MYLIP",
            "RWDD2A",
            "ACPP",
            "TRAF3IP3",
            "ACAP1",
            "ARHGAP15",
            "CST7",
            "P2RY10",
            "SIRPG",
            "GRAP2",
            "FGR",
            "ITGAL",
            "CEACAM21",
            "CD4",
            "BTK",
            "TYROB",
        ],
    },
    'Collison': {
        'Exocrine-like': [
            "REG1B",
            "REG3A",
            "REG1A",
            "PNLIPRP2",
            "CEL",
            "PNLIP",
            "PLA2G1B",
            "CELA3A",
            "CPB1",
            "CELA3B",
            "CTRB2",
            "CLPS",
            "CELA2B",
            "PRSS2",
            "PRSS1",
            "GP2",
            "SLC3A1",
            "CFTR",
            "SLC4A4",
            "SPINK",
        ],
        'Quasi-Mesanchymal': [
            "AIM2",
            "CALHM6",
            "GPM6B",
            "S100A2",
            "KRT14",
            "CAV1",
            "LOX",
            "SLC2A3",
            "TWIST1",
            "PAPPA",
            "NT5E",
            "CKS2",
            "HMMR",
            "SLC5A3",
            "PMAIP1",
            "PHLDA1",
            "SLC16A",
        ],
        'Classical': [
            "FERMT1",
            "HK2",
            "AHNAK2",
            "TMEM45B",
            "SDR16C5",
            "GPRC5A",
            "AGR2",
            "S100P",
            "FXYD3",
            "ST6GALNAC1",
            "CEACAM5",
            "CEACAM6",
            "TFF1",
            "TFF3",
            "CAPN8",
            "FOXQ1",
            "ELF3",
            "ERBB3",
            "TSPAN8",
            "TOX3",
            "LGALS4",
            "PLS1",
            "GPX2",
            "ATP10B",
            "MUC1",
        ]
    },
    'Moffit': {
        'Basal-like': [
            "S100A2",
            "KRT6A",
            "CST6",
            "GPR87",
            "SCEL",
            "ANXA8L2",
            "KRT6C",
            "SERPINB4",
            "SERPINB3",
            "LY6D",
            "LY6D",
            "PLAG1",
            "IL20RB",
            "C16orf74",
            "DCBLD2",
            "KRT17P3",
            "HMGA2",
            "SPRR3",
            "SPRR1B",
            "KRT1",
        ],
        'Classical': [
            "TM4SF5",
            "TFF1",
            "PLA2G10",
            "GPX2",
            "SPINK4",
            "LOC400573",
            "BTNL8",
            "DMBT1",
            "ATAD4",
            "TFF3",
            "FAM3D",
            "LYZ",
            "MYO1A",
            "ANXA10",
            "CLRN3",
            "AKR1B10",
            "CTSE",
            "TSPAN8",
            "LGALS4",
            "REG",
        ]
    }
}

In [102]:
data = []
for k1, d in gene_set_map.items():
    for k2, genes in d.items():
        data.append([k1, k2, ', '.join(genes)])
subtype_genes_df = pd.DataFrame(data=data, columns=['Study', 'Subtype', 'Genes'])
subtype_genes_df

,Study,Subtype,Genes
0,Bailey,ADEX,"MTMR7, ARX, ABCC8, CEACAM7, CACNA2D2, MAPK8IP2..."
1,Bailey,Squamous,"CFH, CD99, HECW1, PLXND1, RECQL, C1orf112, M6P..."
2,Bailey,Pancreatic Progenitor,"ENPP4, CFTR, CYP51A1, ABP1, LASP1, TMEM176A, I..."
3,Bailey,Immunogenic,"CD38, PDK4, FBXL3, CD79B, MYLIP, RWDD2A, ACPP,..."
4,Collison,Exocrine-like,"REG1B, REG3A, REG1A, PNLIPRP2, CEL, PNLIP, PLA..."
5,Collison,Quasi-Mesanchymal,"AIM2, CALHM6, GPM6B, S100A2, KRT14, CAV1, LOX,..."
6,Collison,Classical,"FERMT1, HK2, AHNAK2, TMEM45B, SDR16C5, GPRC5A,..."
7,Moffit,Basal-like,"S100A2, KRT6A, CST6, GPR87, SCEL, ANXA8L2, KRT..."
8,Moffit,Classical,"TM4SF5, TFF1, PLA2G10, GPX2, SPINK4, LOC400573..."


In [103]:
subtype_genes_df.to_csv(os.path.join(save_dir, 'malignant_subtyping_genes.txt'), sep='\t', index=False)

In [99]:
Counter(adata.obs['sample_id'])

Counter({'wu01': 1059,
         'wu02': 1992,
         'wu03': 1721,
         'wu04': 113,
         'wu05': 689,
         'wu07': 3994,
         'wu09': 115,
         'wu14': 1197,
         'wu15': 1158,
         'wu16': 2253,
         'wu18': 1093,
         'wu22': 544,
         'wu23': 246,
         'wu24': 386,
         'wu25': 734,
         'wu26': 1821,
         'wu27': 1806,
         'wu28': 2414,
         'wu29': 1500,
         'wu30': 1820,
         'wu31': 850,
         'wu32': 808,
         '1555-tumor': 3042,
         'H_ZY-1105-07': 5421,
         'H_ZY-1160-06': 1684,
         'H_ZY-1174-06': 12493,
         'TWCE-HT055P-S1PAG': 3611,
         'TWCE-HT081P1-XB1': 3994,
         'T1': 1137,
         'T2': 3007,
         'T3': 1293,
         'T4': 1010,
         'T5': 1102,
         'T6': 1852,
         'T7': 741,
         'T8': 670,
         'T9': 2350,
         'T10': 820,
         'T11': 3090,
         'T12': 2227,
         'T13': 2044,
         'T14': 1945,
         'T15

In [17]:
cytotrace = pd.read_csv('/diskmnt/Projects/Users/estorrs/sc_pdac/data/cytotrace/05242021/results.txt',
                       sep='\t')
# cytotrace.index = [x[:-2] + '-' + x[-1] for x in cytotrace.index]
cytotrace.index = [x.replace('.', '-') if 'h5.h5' not in x else x for x in cytotrace.index]
cytotrace.index = [x[:-2] + '-' + x[-1] if 'h5.h5' in x else x for x in cytotrace.index]
cytotrace.index = [x[1:] if 'X'==x[0] else x for x in cytotrace.index]

cytotrace.columns = ['Cytotrace']
cytotrace

,Cytotrace
100070_filtered_gene_bc_matrices_h5.h5_AAACCTGAGCGTAATA-1,0.761194
100070_filtered_gene_bc_matrices_h5.h5_AAACCTGCAATCCGAT-1,0.717413
100070_filtered_gene_bc_matrices_h5.h5_AAACCTGCACTAGTAC-1,0.944279
100070_filtered_gene_bc_matrices_h5.h5_AAACGGGAGACTGGGT-1,0.744279
100070_filtered_gene_bc_matrices_h5.h5_AAACGGGAGGGTCGAT-1,0.813930
...,...
TWCE-HT081P1-XB1_TTTCCTCTCGTGTTCC-1,0.232836
TWCE-HT081P1-XB1_TTTGACTCAACTCGTA-1,0.154229
TWCE-HT081P1-XB1_TTTGACTTCGGCTTGG-1,0.334328
TWCE-HT081P1-XB1_TTTGGAGGTTGTCATG-1,0.282587


In [18]:
cytotrace.to_csv(os.path.join(save_dir, 'malignant_cytotrace.txt'), sep='\t')

In [109]:
cytotrace_corr = pd.read_csv('/diskmnt/Projects/Users/estorrs/sc_pdac/data/cytotrace/05242021/correlations.txt',
                       sep='\t')
cytotrace_corr

,x
A1BG,-0.023803
A1CF,-0.020353
A2M,-0.232380
A2ML1,0.055582
A4GALT,0.140970
...,...
ZXDB,0.055276
ZXDC,0.097171
ZYG11B,0.056793
ZYX,0.142694


In [110]:
cytotrace_corr.to_csv(os.path.join(save_dir, 'malignant_cytotrace_gene_correlations.txt'), sep='\t')

In [19]:
integrated = sc.read_h5ad('../results/survival/05242021/integrated_v2.h5ad')
integrated

AnnData object with n_obs × n_vars = 391 × 21
    obs: 'age', 'gender', 'race', 'tumor_stage', 'survival_time', 'status', 'dataset', 'Malignant_0_tumor_fraction', 'Malignant_1_tumor_fraction', 'Malignant_2_tumor_fraction', 'Malignant_3_tumor_fraction', 'Malignant_4_tumor_fraction', 'leiden'
    uns: 'dataset_colors', "dendrogram_['leiden']", 'leiden_colors', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'

In [22]:
keep = ['age', 'gender', 'race', 'tumor_stage', 'survival_time', 'status', 'dataset', 'leiden']
integrated.obs = integrated.obs[keep]
integrated.obs

,age,gender,race,tumor_stage,survival_time,status,dataset,leiden
SA518851,67.0,male,unknown,unknown,5,2,bailey,4
SA528766,61.0,female,unknown,unknown,8,2,bailey,0
SA528763,52.0,male,unknown,unknown,36,2,bailey,2
SA518854,55.0,male,unknown,unknown,5,1,bailey,2
SA518806,51.0,female,unknown,unknown,14,2,bailey,2
...,...,...,...,...,...,...,...,...
T_11_02_A148a,68.0,male,white,unknown,20,2,kirby,0
T_11_04_A026b,64.0,female,white,unknown,49,1,kirby,5
T_11_05_A018b,76.0,male,white,unknown,36,2,kirby,6
T_11_06_A011a,68.0,female,white,unknown,10,2,kirby,0


In [23]:
names = {
    '0': 'Classical Low',
    '1': 'ADEX',
    '2': 'CAF High',
    '3': 'Basal/Classical',
    '4': 'Basal',
    '5': 'Mixed - Immune High',
    '6': 'Classical High',
}
integrated.obs['community'] = [names[x] for x in integrated.obs['leiden']]
integrated.obs

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,age,gender,race,tumor_stage,survival_time,status,dataset,leiden,community
SA518851,67.0,male,unknown,unknown,5,2,bailey,4,Basal
SA528766,61.0,female,unknown,unknown,8,2,bailey,0,Classical Low
SA528763,52.0,male,unknown,unknown,36,2,bailey,2,CAF High
SA518854,55.0,male,unknown,unknown,5,1,bailey,2,CAF High
SA518806,51.0,female,unknown,unknown,14,2,bailey,2,CAF High
...,...,...,...,...,...,...,...,...,...
T_11_02_A148a,68.0,male,white,unknown,20,2,kirby,0,Classical Low
T_11_04_A026b,64.0,female,white,unknown,49,1,kirby,5,Mixed - Immune High
T_11_05_A018b,76.0,male,white,unknown,36,2,kirby,6,Classical High
T_11_06_A011a,68.0,female,white,unknown,10,2,kirby,0,Classical Low


In [36]:
integrated.obs.to_csv(os.path.join(save_dir, 'clinical_bulk_dataset_metadata.txt'), sep='\t')

In [112]:
integrated

AnnData object with n_obs × n_vars = 391 × 21
    obs: 'age', 'gender', 'race', 'tumor_stage', 'survival_time', 'status', 'dataset', 'leiden', 'community'
    uns: 'dataset_colors', "dendrogram_['leiden']", 'leiden_colors', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'

In [111]:
integrated.write_h5ad(os.path.join(save_dir, 'community_adata.h5ad'))

/diskmnt/Projects/Users/estorrs/miniconda3/envs/seurat_v4/lib/python3.7/site-packages/anndata/_core/anndata.py:1212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
... storing 'community' as categorical


In [87]:
bulk_deconv = pd.read_csv('../results/survival/05242021/all_plus_eus_pseudo.tsv', sep='\t', index_col=0)
bulk_deconv.columns = [f'CIBERSORTx {c} fraction' for c in bulk_deconv.columns]
bulk_deconv

,CIBERSORTx Acinar fraction,CIBERSORTx B cell fraction,CIBERSORTx CAF fraction,CIBERSORTx CD4 T cell fraction,CIBERSORTx CD8 T cell fraction,CIBERSORTx CD4/CD8 T cell - exhausted fraction,CIBERSORTx DC fraction,CIBERSORTx Endothelial fraction,CIBERSORTx Erythrocyte fraction,CIBERSORTx Malignant - Classical Low fraction,...,CIBERSORTx Malignant - Basal fraction,CIBERSORTx Malignant - Mixed fraction,CIBERSORTx Malignant - ADEX fraction,CIBERSORTx Mast fraction,CIBERSORTx Monocyte fraction,CIBERSORTx NK fraction,CIBERSORTx Plasma fraction,CIBERSORTx Stellate fraction,CIBERSORTx Treg fraction,CIBERSORTx dataset fraction
wu01,0.000067,0.049889,0.148100,0.098574,0.000000,0.026366,0.082429,0.000045,0.043452,0.000265,...,0.009412,0.010570,0.013547,0.132944,0.264703,0.091824,0.009195,0.007145,0.000000,eus pseudo
wu02,0.000004,0.077198,0.004083,0.077402,0.044808,0.035114,0.249363,0.005196,0.027542,0.044176,...,0.036678,0.019382,0.016462,0.096006,0.173096,0.068045,0.005937,0.013414,0.000000,eus pseudo
wu03,0.001629,0.000000,0.000000,0.021852,0.080348,0.051836,0.048720,0.000357,0.085153,0.240589,...,0.043232,0.000000,0.018073,0.116379,0.136612,0.065944,0.000000,0.000000,0.000000,eus pseudo
wu04,0.000179,0.012850,0.000006,0.064243,0.000000,0.061144,0.028692,0.000144,0.026369,0.396892,...,0.086102,0.014312,0.025741,0.155164,0.000000,0.047345,0.000000,0.000000,0.000000,eus pseudo
wu05,0.000308,0.000000,0.000000,0.033000,0.000000,0.039659,0.081814,0.000090,0.038782,0.304592,...,0.038195,0.000000,0.000000,0.124495,0.042597,0.029725,0.000000,0.000000,0.027636,eus pseudo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_11_02_A148a,0.003152,0.061532,0.064676,0.029957,0.020903,0.030236,0.011758,0.019295,0.018276,0.215197,...,0.035148,0.096721,0.046625,0.010623,0.025551,0.073956,0.005231,0.020434,0.018599,kirby
T_11_04_A026b,0.000431,0.107861,0.059038,0.000000,0.045938,0.021208,0.034816,0.044107,0.020195,0.118582,...,0.016158,0.019073,0.053060,0.000000,0.051150,0.071523,0.060428,0.046672,0.085364,kirby
T_11_05_A018b,0.035875,0.040466,0.072274,0.040187,0.000558,0.027236,0.002615,0.017259,0.016496,0.172285,...,0.036772,0.012458,0.076532,0.005498,0.032049,0.091486,0.015341,0.016100,0.036964,kirby
T_11_06_A011a,0.004866,0.173547,0.061743,0.000000,0.035965,0.002228,0.097064,0.019898,0.016130,0.156820,...,0.042015,0.027614,0.027652,0.000000,0.028066,0.088266,0.006883,0.015629,0.101702,kirby


In [88]:
bulk_deconv = pd.merge(integrated.obs[['community']], bulk_deconv, left_index=True, right_index=True)
bulk_deconv

,community,CIBERSORTx Acinar fraction,CIBERSORTx B cell fraction,CIBERSORTx CAF fraction,CIBERSORTx CD4 T cell fraction,CIBERSORTx CD8 T cell fraction,CIBERSORTx CD4/CD8 T cell - exhausted fraction,CIBERSORTx DC fraction,CIBERSORTx Endothelial fraction,CIBERSORTx Erythrocyte fraction,...,CIBERSORTx Malignant - Basal fraction,CIBERSORTx Malignant - Mixed fraction,CIBERSORTx Malignant - ADEX fraction,CIBERSORTx Mast fraction,CIBERSORTx Monocyte fraction,CIBERSORTx NK fraction,CIBERSORTx Plasma fraction,CIBERSORTx Stellate fraction,CIBERSORTx Treg fraction,CIBERSORTx dataset fraction
SA518851,Basal,0.001829,0.045292,0.184032,0.059004,0.000000,0.029009,0.000000,0.010474,0.002339,...,0.297218,0.049259,0.013157,0.012588,0.009185,0.057373,0.000121,0.005766,0.105584,bailey
SA528766,Classical Low,0.012772,0.090721,0.019602,0.000000,0.046757,0.027078,0.000000,0.003788,0.000388,...,0.099494,0.000000,0.103603,0.000000,0.032293,0.057806,0.000000,0.009528,0.110626,bailey
SA528763,CAF High,0.002718,0.042581,0.248410,0.000000,0.076992,0.000000,0.001299,0.054353,0.000000,...,0.000000,0.000000,0.012370,0.000000,0.037447,0.056987,0.009044,0.006246,0.132687,bailey
SA518854,CAF High,0.130321,0.048956,0.145984,0.010060,0.020539,0.014988,0.000000,0.023105,0.000000,...,0.055755,0.036575,0.020152,0.000000,0.041855,0.056939,0.000000,0.026713,0.104766,bailey
SA518806,CAF High,0.002602,0.038245,0.123989,0.000000,0.040949,0.012734,0.000000,0.017046,0.000000,...,0.099273,0.014060,0.028969,0.000000,0.027625,0.055637,0.002925,0.011739,0.142231,bailey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_11_02_A148a,Classical Low,0.003152,0.061532,0.064676,0.029957,0.020903,0.030236,0.011758,0.019295,0.018276,...,0.035148,0.096721,0.046625,0.010623,0.025551,0.073956,0.005231,0.020434,0.018599,kirby
T_11_04_A026b,Mixed - Immune High,0.000431,0.107861,0.059038,0.000000,0.045938,0.021208,0.034816,0.044107,0.020195,...,0.016158,0.019073,0.053060,0.000000,0.051150,0.071523,0.060428,0.046672,0.085364,kirby
T_11_05_A018b,Classical High,0.035875,0.040466,0.072274,0.040187,0.000558,0.027236,0.002615,0.017259,0.016496,...,0.036772,0.012458,0.076532,0.005498,0.032049,0.091486,0.015341,0.016100,0.036964,kirby
T_11_06_A011a,Classical Low,0.004866,0.173547,0.061743,0.000000,0.035965,0.002228,0.097064,0.019898,0.016130,...,0.042015,0.027614,0.027652,0.000000,0.028066,0.088266,0.006883,0.015629,0.101702,kirby


In [89]:
from mgitools.os_helpers import listfiles

In [90]:
def get_cell_state_df(base_dir, cell_type, dir_names, name=None):
    common = cell_type.replace('.', ' ') if name is None else name
#     fps = sorted(listfiles(base_dir, regex=r'state_assignment.txt'))
#     fps = [fp for fp in fps if cell_type in fp]
#     fps = [fp for fp in fps if any([n in fp for n in dir_names])]
#     dfs = [pd.read_csv(fp, sep='\t', index_col=0) if 'denovo' not in fp else pd.read_csv(fp, sep='\t', index_col=1)
#            for fp in fps]
#     df_as = pd.concat(dfs, axis=0)
#     df_as = df_as[['State']]
#     df_as.columns = [f'Ecotyper {common} state']
    
    fps = sorted(listfiles(base_dir, regex=r'abundances.txt'))
    fps = [fp for fp in fps if cell_type in fp]
    fps = [fp for fp in fps if any([n in fp for n in dir_names])]
    dfs = [pd.read_csv(fp, sep='\t', index_col=0)
           for fp in fps]
    
    dfs = [df.transpose() for df in dfs]
    states = dfs[0].columns
    df_ab = pd.concat(dfs, axis=0)
    df_ab.columns = [f'Ecotyper {common} state {s} abundance' for s in df_ab.columns]
    df_ab[f'Ecotyper {common} state max abundance'] = [states[i] for i in np.argmax(df_ab.values, axis=1)]
#     df = pd.merge(df_as, df_ab, how='right', left_index=True, right_index=True)
    
    return df_ab

def get_ecotype_df(base_dir, dir_names):
    fps = sorted(listfiles(base_dir, regex=r'abundance.txt'))
    fps = [fp for fp in fps if 'ecotype' in fp]
    fps = [fp for fp in fps if any([n in fp for n in dir_names])]
    dfs = [pd.read_csv(fp, sep='\t', index_col=0)
           for fp in fps]
    
    dfs = [df.transpose() for df in dfs]
    states = dfs[0].columns
    df_ab = pd.concat(dfs, axis=0)
    df_ab.columns = [f'Ecotyper ecotype {s} abundance' for s in df_ab.columns]
    df_ab[f'Ecotyper ecotype max abundance'] = [states[i] for i in np.argmax(df_ab.values, axis=1)]
#     df = pd.merge(df_as, df_ab, how='right', left_index=True, right_index=True)
    
    return df_ab

df1 = get_cell_state_df('../data/ecotyper/results', 'CD8.T.cell',
                       ['denovo_discovery', 'bulk_recovery_bailey', 'bulk_recovery_kirby', 'bulk_recovery_moffit'])
df2 = get_cell_state_df('../data/ecotyper/results', 'Malignant',
                       ['denovo_discovery', 'bulk_recovery_bailey', 'bulk_recovery_kirby', 'bulk_recovery_moffit'])
df3 = get_ecotype_df('../data/ecotyper/results',
                    ['denovo_discovery', 'bulk_recovery_bailey', 'bulk_recovery_kirby', 'bulk_recovery_moffit'])
combined_bulk_ab = pd.merge(df1, df2, left_index=True, right_index=True)
combined_bulk_ab = pd.merge(combined_bulk_ab, df3, left_index=True, right_index=True)
bulk_deconv = pd.merge(bulk_deconv, combined_bulk_ab, how='left', left_index=True, right_index=True)
bulk_deconv

,community,CIBERSORTx Acinar fraction,CIBERSORTx B cell fraction,CIBERSORTx CAF fraction,CIBERSORTx CD4 T cell fraction,CIBERSORTx CD8 T cell fraction,CIBERSORTx CD4/CD8 T cell - exhausted fraction,CIBERSORTx DC fraction,CIBERSORTx Endothelial fraction,CIBERSORTx Erythrocyte fraction,...,Ecotyper Malignant state S04 abundance,Ecotyper Malignant state S05 abundance,Ecotyper Malignant state max abundance,Ecotyper ecotype E1 abundance,Ecotyper ecotype E2 abundance,Ecotyper ecotype E3 abundance,Ecotyper ecotype E4 abundance,Ecotyper ecotype E5 abundance,Ecotyper ecotype E6 abundance,Ecotyper ecotype max abundance
SA518851,Basal,0.001829,0.045292,0.184032,0.059004,0.000000,0.029009,0.000000,0.010474,0.002339,...,2.186537e-15,2.186537e-15,S01,0.168398,0.058443,0.277484,0.050577,0.217020,0.228079,E3
SA528766,Classical Low,0.012772,0.090721,0.019602,0.000000,0.046757,0.027078,0.000000,0.003788,0.000388,...,7.322485e-02,2.571821e-01,S01,0.124176,0.297399,0.188038,0.317364,0.005100,0.067923,E4
SA528763,CAF High,0.002718,0.042581,0.248410,0.000000,0.076992,0.000000,0.001299,0.054353,0.000000,...,2.131513e-01,3.349138e-01,S05,0.203161,0.258509,0.070067,0.299092,0.099903,0.069268,E4
SA518854,CAF High,0.130321,0.048956,0.145984,0.010060,0.020539,0.014988,0.000000,0.023105,0.000000,...,8.000525e-02,3.421170e-15,S01,0.201189,0.088875,0.239492,0.100028,0.278130,0.092286,E5
SA518806,CAF High,0.002602,0.038245,0.123989,0.000000,0.040949,0.012734,0.000000,0.017046,0.000000,...,6.697391e-06,4.049557e-02,S01,0.115322,0.119323,0.259675,0.233329,0.225189,0.047161,E3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_11_02_A148a,Classical Low,0.003152,0.061532,0.064676,0.029957,0.020903,0.030236,0.011758,0.019295,0.018276,...,1.784887e-01,1.398609e-01,S03,0.043937,0.141588,0.184958,0.109020,0.317657,0.202841,E5
T_11_04_A026b,Mixed - Immune High,0.000431,0.107861,0.059038,0.000000,0.045938,0.021208,0.034816,0.044107,0.020195,...,1.488872e-01,1.553184e-01,S05,0.339599,0.081628,0.030431,0.204724,0.182628,0.160989,E1
T_11_05_A018b,Classical High,0.035875,0.040466,0.072274,0.040187,0.000558,0.027236,0.002615,0.017259,0.016496,...,1.905203e-01,3.810862e-01,S05,0.108861,0.131807,0.128387,0.267432,0.185866,0.177648,E4
T_11_06_A011a,Classical Low,0.004866,0.173547,0.061743,0.000000,0.035965,0.002228,0.097064,0.019898,0.016130,...,7.246253e-11,1.978764e-01,S01,0.352015,0.076926,0.108164,0.220193,0.050510,0.192191,E1


In [91]:
bulk_deconv.to_csv(os.path.join(save_dir, 'bulk_cibersortx_and_ecotyper.txt'), sep='\t')

In [29]:
df = pd.read_csv('/diskmnt/Projects/Users/estorrs/sc_pdac/data/ecotyper/results/summarized_pseudo.txt', sep='\t',
                index_col=0)
df

,overall_survival,status,community,CIBERSORTx Acinar fraction,CIBERSORTx B cell fraction,CIBERSORTx CAF fraction,CIBERSORTx CD4 T cell fraction,CIBERSORTx CD8 T cell fraction,CIBERSORTx CD4/CD8 T cell - exhausted fraction,CIBERSORTx DC fraction,...,Ecotyper CD8 T cell state,Ecotyper CD8 T cell state S01 abundance,Ecotyper CD8 T cell state S02 abundance,Ecotyper CD8 T cell state S03 abundance,Ecotyper Malignant state,Ecotyper Malignant state S01 abundance,Ecotyper Malignant state S02 abundance,Ecotyper Malignant state S03 abundance,Ecotyper Malignant state S04 abundance,Ecotyper Malignant state S05 abundance
sample_id,,,,,,,,,,,,,,,,,,,,,
wu01,2.0,Dead,CAF High,0.000067,0.049889,0.148100,0.098574,0.000000,0.026366,0.082429,...,S01,3.392896e-01,2.680908e-01,2.847413e-01,NaN,2.167996e-02,2.912954e-15,2.912954e-15,2.912954e-15,1.885867e-02
wu02,1.0,Dead,Basal,0.000004,0.077198,0.004083,0.077402,0.044808,0.035114,0.249363,...,NaN,1.400904e-01,4.593479e-15,1.174348e-01,NaN,2.968282e-15,1.332010e-02,8.737192e-10,2.968282e-15,2.968282e-15
wu03,0.0,Dead,Classical Low,0.001629,0.000000,0.000000,0.021852,0.080348,0.051836,0.048720,...,S02,1.910391e-01,5.170655e-01,2.293641e-01,S03,7.584739e-02,1.544481e-15,3.643826e-01,2.513002e-01,3.084698e-01
wu04,20.0,Dead,Classical Low,0.000179,0.012850,0.000006,0.064243,0.000000,0.061144,0.028692,...,NaN,1.974917e-15,1.974917e-15,9.105888e-02,NaN,7.484072e-02,8.674499e-03,1.345944e-02,1.779800e-15,1.779800e-15
wu05,35.0,Alive,Classical Low,0.000308,0.000000,0.000000,0.033000,0.000000,0.039659,0.081814,...,NaN,4.924772e-15,4.924772e-15,8.089047e-02,S04,9.565007e-07,2.776025e-01,2.430386e-01,3.310847e-01,3.262544e-15
wu07,5.0,Dead,Basal,0.000585,0.118269,0.015033,0.174981,0.035516,0.025192,0.082507,...,S02,3.161887e-01,5.234855e-01,1.490707e-01,S01,4.505488e-01,2.277875e-15,1.720850e-01,1.447814e-08,1.573352e-01
wu09,8.0,Dead,Classical Low,0.001436,0.007692,0.000000,0.000107,0.000000,0.051568,0.018408,...,NaN,1.836634e-15,1.836634e-15,6.211821e-02,NaN,3.936626e-02,1.492712e-02,1.766809e-02,1.831179e-15,4.325010e-06
wu14,29.0,Alive,Mixed - Immune High,0.000041,0.041424,0.059354,0.120918,0.075470,0.015946,0.068149,...,NaN,8.600641e-02,6.054912e-15,1.406103e-01,NaN,3.723697e-15,3.081701e-01,1.009047e-01,8.025129e-02,1.974432e-02
wu15,29.0,Alive,Classical Low,0.000095,0.093825,0.001656,0.454125,0.057154,0.000000,0.187316,...,S01,5.960415e-01,1.007973e-01,6.145799e-15,NaN,3.244768e-02,3.051710e-04,4.238042e-15,4.238042e-15,1.544214e-01


In [30]:
keep = [c for c in df.columns if c not in ['overall_survival', 'status']]
df = df[keep]
df

,community,CIBERSORTx Acinar fraction,CIBERSORTx B cell fraction,CIBERSORTx CAF fraction,CIBERSORTx CD4 T cell fraction,CIBERSORTx CD8 T cell fraction,CIBERSORTx CD4/CD8 T cell - exhausted fraction,CIBERSORTx DC fraction,CIBERSORTx Endothelial fraction,CIBERSORTx Erythrocyte fraction,...,Ecotyper CD8 T cell state,Ecotyper CD8 T cell state S01 abundance,Ecotyper CD8 T cell state S02 abundance,Ecotyper CD8 T cell state S03 abundance,Ecotyper Malignant state,Ecotyper Malignant state S01 abundance,Ecotyper Malignant state S02 abundance,Ecotyper Malignant state S03 abundance,Ecotyper Malignant state S04 abundance,Ecotyper Malignant state S05 abundance
sample_id,,,,,,,,,,,,,,,,,,,,,
wu01,CAF High,0.000067,0.049889,0.148100,0.098574,0.000000,0.026366,0.082429,0.000045,0.043452,...,S01,3.392896e-01,2.680908e-01,2.847413e-01,NaN,2.167996e-02,2.912954e-15,2.912954e-15,2.912954e-15,1.885867e-02
wu02,Basal,0.000004,0.077198,0.004083,0.077402,0.044808,0.035114,0.249363,0.005196,0.027542,...,NaN,1.400904e-01,4.593479e-15,1.174348e-01,NaN,2.968282e-15,1.332010e-02,8.737192e-10,2.968282e-15,2.968282e-15
wu03,Classical Low,0.001629,0.000000,0.000000,0.021852,0.080348,0.051836,0.048720,0.000357,0.085153,...,S02,1.910391e-01,5.170655e-01,2.293641e-01,S03,7.584739e-02,1.544481e-15,3.643826e-01,2.513002e-01,3.084698e-01
wu04,Classical Low,0.000179,0.012850,0.000006,0.064243,0.000000,0.061144,0.028692,0.000144,0.026369,...,NaN,1.974917e-15,1.974917e-15,9.105888e-02,NaN,7.484072e-02,8.674499e-03,1.345944e-02,1.779800e-15,1.779800e-15
wu05,Classical Low,0.000308,0.000000,0.000000,0.033000,0.000000,0.039659,0.081814,0.000090,0.038782,...,NaN,4.924772e-15,4.924772e-15,8.089047e-02,S04,9.565007e-07,2.776025e-01,2.430386e-01,3.310847e-01,3.262544e-15
wu07,Basal,0.000585,0.118269,0.015033,0.174981,0.035516,0.025192,0.082507,0.000167,0.041239,...,S02,3.161887e-01,5.234855e-01,1.490707e-01,S01,4.505488e-01,2.277875e-15,1.720850e-01,1.447814e-08,1.573352e-01
wu09,Classical Low,0.001436,0.007692,0.000000,0.000107,0.000000,0.051568,0.018408,0.000449,0.005197,...,NaN,1.836634e-15,1.836634e-15,6.211821e-02,NaN,3.936626e-02,1.492712e-02,1.766809e-02,1.831179e-15,4.325010e-06
wu14,Mixed - Immune High,0.000041,0.041424,0.059354,0.120918,0.075470,0.015946,0.068149,0.000013,0.023542,...,NaN,8.600641e-02,6.054912e-15,1.406103e-01,NaN,3.723697e-15,3.081701e-01,1.009047e-01,8.025129e-02,1.974432e-02
wu15,Classical Low,0.000095,0.093825,0.001656,0.454125,0.057154,0.000000,0.187316,0.000021,0.009178,...,S01,5.960415e-01,1.007973e-01,6.145799e-15,NaN,3.244768e-02,3.051710e-04,4.238042e-15,4.238042e-15,1.544214e-01


In [31]:
df.to_csv(os.path.join(save_dir, 'pseudo_bulk_cibersortx_and_ecotyper_results.txt'), sep='\t')

In [104]:
pathways = pd.read_csv('../results/figures/ecotyper_v1/cd8_t_cell_s01_pathway.txt', sep='\t')
pathways['-log10(FDR)'] = [-np.log10(x) for x in pathways['q-value FDR B&H']]
pathways['Genes in pathway'] = pathways['Hit Count in Query List'].to_list()
# pathways['NK'] = go_pathways
pathways

,Category,ID,Name,Source,p-value,q-value Bonferroni,q-value FDR B&H,q-value FDR B&Y,Hit Count in Query List,Hit Count in Genome,Hit in Query List,-log10(FDR),Genes in pathway
0,GO: Biological Process,GO:0050851,antigen receptor-mediated signaling pathway,,1.200000e-10,1.264000e-07,1.264000e-07,9.526000e-07,8,338,"SH2D1A,TRAT1,FYN,CD28,CD3D,CCR7,ITK,FCRL3",6.898253,8
1,GO: Biological Process,GO:0050852,T cell receptor signaling pathway,,2.824000e-10,2.974000e-07,1.487000e-07,1.121000e-06,7,219,"SH2D1A,TRAT1,FYN,CD28,CD3D,CCR7,ITK",6.827689,7
2,GO: Biological Process,GO:0050776,regulation of immune response,,2.213000e-09,2.330000e-06,4.828000e-07,3.639000e-06,10,1066,"SELL,SH2D1A,TRAT1,FYN,CD28,CD3D,CCR7,ITK,FCRL3...",6.316233,10
3,GO: Biological Process,GO:0002429,immune response-activating cell surface recept...,,2.568000e-09,2.704000e-06,4.828000e-07,3.639000e-06,8,498,"SH2D1A,TRAT1,FYN,CD28,CD3D,CCR7,ITK,FCRL3",6.316233,8
4,GO: Biological Process,GO:0002757,immune response-activating signal transduction,,2.608000e-09,2.747000e-06,4.828000e-07,3.639000e-06,8,499,"SH2D1A,TRAT1,FYN,CD28,CD3D,CCR7,ITK,FCRL3",6.316233,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,GO: Biological Process,GO:0010310,regulation of hydrogen peroxide metabolic process,,1.918000e-02,1.000000e+00,4.796000e-02,3.615000e-01,1,23,FYN,1.319121,1
421,GO: Biological Process,GO:0008037,cell recognition,,1.960000e-02,1.000000e+00,4.890000e-02,3.685000e-01,2,259,"CCR7,CCL19",1.310691,2
422,GO: Biological Process,GO:0050966,detection of mechanical stimulus involved in s...,,2.000000e-02,1.000000e+00,4.956000e-02,3.735000e-01,1,24,FYN,1.304869,1
423,GO: Biological Process,GO:0002922,positive regulation of humoral immune response,,2.000000e-02,1.000000e+00,4.956000e-02,3.735000e-01,1,24,CCR7,1.304869,1


In [105]:
pathways.to_csv(os.path.join(save_dir, 'cd8_s01_gsea.txt'), sep='\t', index=False)

In [106]:
df = pd.read_csv('../data/ecotyper/results/denovo_discovery/CD8.T.cell/gene_info.txt', sep='\t')
df

,Gene,IS01,IS02,IS03,IS04,IS05,State,MaxFC,InitialState
SH2D1A,SH2D1A,-0.137263,-0.363214,-1.035431,1.727172,-0.148235,S01,1.727172,IS04
AC243960,AC243960,-0.148461,-0.109804,-0.909496,1.723809,-0.516182,S01,1.723809,IS04
CD28,CD28,0.079854,-0.673154,-1.070901,1.682430,0.003241,S01,1.682430,IS04
LINC00861,LINC00861,0.014843,-0.225026,-0.865906,1.678200,-0.581588,S01,1.678200,IS04
ITK,ITK,0.019809,-0.702542,-0.810925,1.674511,-0.211798,S01,1.674511,IS04
...,...,...,...,...,...,...,...,...,...
C11orf97,C11orf97,0.175674,0.028753,-0.024355,-0.338312,0.187763,S03,0.187763,IS05
MOB4,MOB4,-0.156956,-0.110348,0.097889,-0.025809,0.172516,S03,0.172516,IS05
GPR19,GPR19,0.048734,-0.053333,-0.121892,-0.017816,0.164334,S03,0.164334,IS05
CEP83,CEP83,-0.087628,0.016277,0.047342,-0.097513,0.123554,S03,0.123554,IS05


In [107]:
df.to_csv(os.path.join(save_dir, 'cd8_cell_state_associated_genes.txt'), sep='\t')